image.png

In [1]:
import tensorflow as tf
import numpy as np # 추가
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report # 추가


2025-11-20 13:34:14.251858: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-20 13:34:16.260709: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-20 13:34:30.596930: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
# 사용자 설정
train_dir = '/workspace/merge_data_binary/new_data/img_binary'
val_dir = '/workspace/merge_data_binary/new_data_val/img_binary'



target_size = (224, 224)
batch_size = 32
seed = 42


In [3]:

# Data generators: EfficientNet 전처리 사용 (preprocess_input)
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # ImageNet 기반 모델용 전처리
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)
# 제너레이터: 이진 분류이므로 class_mode='binary'
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary',
    seed=seed
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False, # 평가 시 순서 유지를 위해 False 설정
    seed=seed
)

print("Class indices:", train_generator.class_indices)
if 'happy' in train_generator.class_indices:
    print("'happy' class index:", train_generator.class_indices['happy'])
    
# Base model 로드
base_model = EfficientNetB0(weights='imagenet', include_top=False,
                           input_shape=(target_size[0], target_size[1], 3))

# top 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # 이진 분류

model = Model(inputs=base_model.input, outputs=predictions)

# base freeze
for layer in base_model.layers:
    layer.trainable = False
    
# 컴파일 (이진 교차엔트로피)
model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

model.summary()


# 안전한 step 계산
steps_per_epoch = len(train_generator)
validation_steps = len(validation_generator)

# 학습: 
print("\n--- 모델 학습 시작 ---")
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_steps,    
)
print("--- 모델 학습 완료 ---\n")

# --- 검증 데이터셋에 대한 성능 평가 지표 추가 ---

print("--- 검증 데이터셋 성능 평가 시작 ---")

# 1. 검증 데이터셋의 실제 레이블 (True Labels) 추출
# validation_generator는 shuffle=False로 설정되어 있으므로 순서가 유지됩니다.
validation_labels = validation_generator.classes

# 2. 모델 예측 (Predictions) 수행
# validation_generator를 사용하여 검증 데이터셋에 대한 예측 확률을 얻습니다.
validation_generator.reset() # 제너레이터 초기화 (중요)
pred_probabilities = model.predict(
    validation_generator, 
    steps=validation_steps,
    verbose=1
)

# 3. 예측 확률을 이진 클래스 (0 또는 1)로 변환
# Sigmoid 출력은 확률이므로, 0.5를 기준으로 클래스를 결정합니다.
# 이진 분류에서는 0.5를 임계값으로 사용합니다.
pred_classes = (pred_probabilities > 0.5).astype(int).flatten()


# 4. 클래스 레이블 매핑 (선택적)
# classification_report 출력을 위해 클래스 이름을 가져옵니다.
class_names = list(validation_generator.class_indices.keys())
print("Class Names for Report:", class_names)


# 5. classification_report를 사용하여 지표 계산 및 출력
# precision, recall, f1-score, support를 포함합니다.
report = classification_report(
    validation_labels, 
    pred_classes, 
    target_names=class_names, 
    digits=4 # 소수점 자릿수 지정
)

print("\n--- Classification Report (Validation Set) ---\n")
print(report)

print("--- 검증 데이터셋 성능 평가 완료 ---")

Found 9550 images belonging to 2 classes.
Found 1151 images belonging to 2 classes.


Class indices: {'happy': 0, 'other': 1}
'happy' class index: 0


I0000 00:00:1763645682.887692  464680 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9157 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:53:00.0, compute capability: 8.6


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

...

 57/299 ━━━━━━━━━━━━━━━━━━━━ 8:06 2s/step - accuracy: 0.5365 - auc: 0.5322 - loss: 0.7222

2025-11-20 13:37:16.995580: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-20 13:37:17.120741: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-20 13:37:17.491737: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-20 13:37:17.618282: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-20 13:37:18.119024: E external/local_xla/xla/stream_

299/299 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5640 - auc: 0.5836 - loss: 0.6866

2025-11-20 13:46:53.586850: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-20 13:46:53.713456: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-20 13:46:54.117390: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-20 13:46:54.243870: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-20 13:46:54.727768: E external/local_xla/xla/stream_

299/299 ━━━━━━━━━━━━━━━━━━━━ 727s 2s/step - accuracy: 0.5986 - auc: 0.6361 - loss: 0.6602 - val_accuracy: 0.7272 - val_auc: 0.6917 - val_loss: 0.5521
Epoch 2/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 610s 2s/step - accuracy: 0.6662 - auc: 0.7162 - loss: 0.6132 - val_accuracy: 0.6533 - val_auc: 0.7100 - val_loss: 0.6080
Epoch 3/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 643s 2s/step - accuracy: 0.6871 - auc: 0.7485 - loss: 0.5873 - val_accuracy: 0.5378 - val_auc: 0.7168 - val_loss: 0.7485
Epoch 4/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 649s 2s/step - accuracy: 0.6933 - auc: 0.7567 - loss: 0.5787 - val_accuracy: 0.6012 - val_auc: 0.7007 - val_loss: 0.6931
Epoch 5/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 644s 2s/step - accuracy: 0.7094 - auc: 0.7769 - loss: 0.5636 - val_accuracy: 0.5282 - val_auc: 0.7216 - val_loss: 0.8218
Epoch 6/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 624s 2s/step - accuracy: 0.7197 - auc: 0.7915 - loss: 0.5470 - val_accuracy: 0.6603 - val_auc: 0.7181 - val_loss: 0.6426
Epoch 7/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 634s 

In [4]:

# 5. classification_report를 사용하여 지표 계산 및 출력
# precision, recall, f1-score, support를 포함합니다.
report = classification_report(
    validation_labels, 
    pred_classes, 
    target_names=class_names, 
    digits=4 # 소수점 자릿수 지정
)

print("\n--- Classification Report (Validation Set) ---\n")
print(report)


--- Classification Report (Validation Set) ---

              precision    recall  f1-score   support

       happy     0.3803    0.6957    0.4917       299
       other     0.8493    0.6021    0.7047       852

    accuracy                         0.6264      1151
   macro avg     0.6148    0.6489    0.5982      1151
weighted avg     0.7275    0.6264    0.6494      1151

